In [1]:
import json
import pickle
# from string import punctuation
import re
import numpy as np

import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def unpack(file):
    with open(file, "r", encoding='utf-8') as f:
        json_list = list(f)

    all_news_text_X = []
    all_news_summ_Y = []
    for i, str in enumerate(json_list):
        one_news = json.loads(str)
        del one_news['url']
        del one_news['title']
        del one_news['date']
        all_news_text_X.append(one_news['text'])
        all_news_summ_Y.append(one_news['summary'])
    return all_news_text_X, all_news_summ_Y

In [3]:
def spliting_list(list, delimiter):
    splitted = [[]]
    for lemma in list:
        if lemma == delimiter:
            splitted.append([])
        else:
            splitted[-1].append(lemma)
    return splitted

In [11]:
def preprocess(text_str, punctuation_re):
    mystem = Mystem()
    rus_stopwords = stopwords.words("russian")

    text_lemmas = mystem.lemmatize(text_str.lower())
    text_lemmas = [punctuation_re.sub('', lemma) for lemma in text_lemmas ] # удаляю все кроме букв и цифр
    text_lemmas = [lemma for lemma in text_lemmas if lemma.strip() != '' and lemma not in rus_stopwords]

    return text_lemmas

In [44]:
def tfidf_bag_words_make_vocab(all_sent_list):
    tf_vectorizer = TfidfVectorizer(max_features=50000, max_df=0.85)  # слово встречается > чем в 85% -> не берем

    all_sent_str = [" ".join(one_sent_list) for one_sent_list in all_sent_list]  # список из предложений=строк
    bag_of_words = tf_vectorizer.fit_transform(all_sent_str)
    vocabulary = tf_vectorizer.get_feature_names()
    return bag_of_words, vocabulary

In [46]:
def score(vec_X, vec_Y):
    # count angle between center vectors
    vec_X_unit = (vec_X / np.linalg.norm(vec_X)) if np.linalg.norm(vec_X) != 0 else np.zeros(vec_X.shape)
    vec_Y_unit = (vec_Y / np.linalg.norm(vec_Y)) if np.linalg.norm(vec_Y) != 0 else np.zeros(vec_Y.shape)
    angles = np.arccos(np.clip(np.dot(vec_X_unit, vec_Y_unit), -1.0, 1.0))
    # привожу [0 - хорошо, pi/2 - плохо] к [0 - плохо, 1 - хорошо]
    score_01 = 1 - (angles / (np.pi/2))
    return score_01

### MAIN

In [7]:
all_news_text_X, all_news_summ_Y = unpack(
        'C:/Users/Acer/PycharmProjects/ML_NLP_course3/gazeta_jsonl/gazeta_train.jsonl')
# my_punctuation = punctuation + '–' + '—' + '«' + '»' + '», —' + '%, ' + '» — ' + ': «' + '» (' + '), ' + ', «'
punctuation_re = re.compile('[^a-zA-Zа-яА-Я0-9]+')

In [15]:
all_preproc_sent_text_X = []
count_sent_in_text_X = []
pack_size = 1000
start_ind = 0  # to count start new 1000 text pack
while start_ind < len(all_news_text_X):
#     # временное ограничение!!!!
#     if start_ind >= 6:
#         break
    
    # join and preprocessing
    pack_list_for_preproc = []
    end_ind = start_ind + pack_size
    if end_ind > len(all_news_text_X):
        end_ind = len(all_news_text_X)
        
    for i in range(start_ind, end_ind):
        sentences = nltk.sent_tokenize(all_news_text_X[i])
        count_sent_in_text_X.append(len(sentences))
        pack_list_for_preproc.append(' bbreakk '.join(sentences))  # between sentences 'bbreakk'
    pack_str_for_preproc = ' bbreakk '.join(pack_list_for_preproc)
    pack_lemmas = preprocess(pack_str_for_preproc, punctuation_re)

    # split
    preproc_sentences = spliting_list(pack_lemmas, 'bbreakk') # список из предложений=строк
    all_preproc_sent_text_X.extend(preproc_sentences)
    
    start_ind += pack_size
    print(end_ind)
    
    if sum(count_sent_in_text_X) != len(all_preproc_sent_text_X):
        print('error')
        break
    
count_sent_in_text_X = np.array(count_sent_in_text_X)    
all_preproc_sent_text_X = np.array(all_preproc_sent_text_X)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
52400


<ipython-input-15-bd6333328061>:41: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  all_preproc_sent_text_X = np.array(all_preproc_sent_text_X)


In [16]:
with open('all_preproc_sent_text_X.pickle', 'wb') as f:
    pickle.dump(all_preproc_sent_text_X, f)
with open('count_sent_in_text_X.pickle', 'wb') as f:
    pickle.dump(count_sent_in_text_X, f)

In [19]:
all_preproc_sent_summ_Y = []
count_sent_in_summ_Y = []
pack_size = 1000
start_ind = 0  # to count start new 1000 text pack
while start_ind < len(all_news_summ_Y):
#     # временное ограничение!!!!
#     if start_ind >= 6:
#         break
    
    # join and preprocessing
    pack_list_for_preproc = []
    end_ind = start_ind + pack_size
    if end_ind > len(all_news_summ_Y):
        end_ind = len(all_news_summ_Y)
        
    for i in range(start_ind, end_ind):
        sentences = nltk.sent_tokenize(all_news_summ_Y[i])
        count_sent_in_summ_Y.append(len(sentences))
        pack_list_for_preproc.append(' bbreakk '.join(sentences))  # between sentences '<break>'
    pack_str_for_preproc = ' bbreakk '.join(pack_list_for_preproc)
    pack_lemmas = preprocess(pack_str_for_preproc, punctuation_re)

    # split
    preproc_sentences = spliting_list(pack_lemmas, 'bbreakk') # список из предложений=строк
    all_preproc_sent_summ_Y.extend(preproc_sentences)
    
    start_ind += pack_size
    print(end_ind)
    
    if sum(count_sent_in_summ_Y) != len(all_preproc_sent_summ_Y):
        print('error')
        break
    
count_sent_in_summ_Y = np.array(count_sent_in_summ_Y)    
all_preproc_sent_summ_Y = np.array(all_preproc_sent_summ_Y)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
52400


<ipython-input-19-be508767569f>:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  all_preproc_sent_summ_Y = np.array(all_preproc_sent_summ_Y)


In [20]:
with open('all_preproc_sent_summ_Y.pickle', 'wb') as f:
    pickle.dump(all_preproc_sent_summ_Y, f)
with open('count_sent_in_summ_Y.pickle', 'wb') as f:
    pickle.dump(count_sent_in_summ_Y, f)

In [51]:
'''
надо обработанные тексты Х У объединить, чтобы сделать общий мешок слов
потом разделить обратно, но каждое предложение будет выглядеть как вектор tf-idf из всеех слов всех текстов и аннотаций
'''
all_sent = np.concatenate((all_preproc_sent_text_X, all_preproc_sent_summ_Y), axis=None)
tfidf_all_sent, train_vocabulary = tfidf_bag_words_make_vocab(all_sent)
print(len(train_vocabulary))

50000


In [53]:
'''
сохранение каркаса мешка слов для разложения по нему вал и тест предложений
'''
print(np.random.permutation(train_vocabulary)[:1000])
with open('Vocabulary.pickle', 'wb') as f:
    pickle.dump(train_vocabulary, f)

['cnn' 's4' 'юрский' 'торстен' 'обманщик' 'склонение' 'подобие' 'привычно'
 'императорский' 'сцепляться' 'q5' 'позавчерашний' 'недорост'
 'дефибриллятор' 'подлинно' 'демограф' 'тальявенто' 'запасник' 'погорилый'
 'флери' 'состряпать' 'коровин' 'искривлять' 'калий' 'кб' 'cw' 'тритэ'
 'изыскивать' 'дипотношение' 'червенок' 'битцевский' 'приставание'
 'министерский' 'конобрий' 'воспроизводство' 'перезаключение'
 'всматриваться' 'долгожданный' 'оккупационный' 'структурировать'
 'сенаторзой' 'залив' 'имэмо' 'преемница' 'фара' 'масальскис'
 'невозвратный' 'мифический' 'повторный' 'убавляться' 'сбыт' 'олигархия'
 'неспособность' 'рукопашная' 'корректно' 'парамонов' 'драгич' 'федерация'
 'жена' 'ведомо' 'пырьев' 'мосгорнаследие' 'or' 'своеобразный'
 'фотохостинг' 'самоустранение' 'мышца' 'безнадежно' 'самара' 'смуляна'
 'месиво' 'франзен' 'awards' 'никакой' 'малочисленность' 'гринуэй'
 'усадьба' 'обкрадывать' 'гарик' 'публично' 'увешать' 'экий' 'ланта'
 'донский' 'ассамблея' 'мина' 'пиганович'

In [54]:
# разделить предложения по текстам, а тексты по наборам X и Y
all_tfidf_X = []
all_tfidf_Y = []
already_taked = 0
for count in count_sent_in_text_X:
    all_tfidf_X.append(tfidf_all_sent[already_taked:already_taked + count])
    already_taked += count
for count in count_sent_in_summ_Y:
    all_tfidf_Y.append(tfidf_all_sent[already_taked:already_taked + count])
    already_taked += count

In [55]:
# получить список предложений набора Х, не разделенных на тексты, для модели
count_sent_in_X = np.sum(count_sent_in_text_X)
all_tfidf_X_no_text = tfidf_all_sent[:count_sent_in_X]

In [56]:
'''
посчитать score для каждого предложения
сравнить каждое предложение с каждым предложением из аннотации, выбрать минимальный score
'''
score_all_sent = []
for text_index in range(len(all_tfidf_X)):
    sentences_X = all_tfidf_X[text_index].toarray()
    sentences_Y = all_tfidf_Y[text_index].toarray()
    for vec_x in sentences_X:
        best_min_score = np.min([score(vec_x, vec_y) for vec_y in sentences_Y])
        score_all_sent.append(best_min_score)
score_all_sent = np.array(score_all_sent)

In [57]:
print(len(np.array(score_all_sent)))
print(sum(np.array(score_all_sent) != 0))

1985691
394490


In [58]:
with open('X_out_train.pickle', 'wb') as f:
    pickle.dump(all_tfidf_X_no_text, f)  # !!!отправляю не toarray(), а sparce matrix
with open('Y_out_train.pickle', 'wb') as f:
    pickle.dump(score_all_sent, f)